In [9]:
import pandas as pd

x = pd.read_pickle('../ml/signal_stat/buy_stat_1h.pkl')
x.sort_values('time')

,time,ticker,timeframe,pattern,signal_price,signal_smooth_price,pct_price_diff_1,mfe_1,mae_1,pct_price_diff_2,...,mae_21,pct_price_diff_22,mfe_22,mae_22,pct_price_diff_23,mfe_23,mae_23,pct_price_diff_24,mfe_24,mae_24
1726,1970-01-20 15:02:20.400,RAY_USDT,1h,STOCH_RSI,0.201000,0.200125,0.124766,1.994810,0.997405,0.290637,...,2.742863,-1.265022,2.493512,2.742863,-1.243676,2.493512,2.742863,-1.350496,2.493512,2.742863
1721,1970-01-20 15:44:20.400,FILECOIN_USDT,1h,STOCH_RSI,3.961000,4.345042,-0.656364,0.102199,0.562094,-1.324355,...,0.987923,-9.174187,3.083000,0.987923,-9.077121,3.083000,0.987923,-8.965424,3.083000,0.987923
1720,1970-01-20 15:44:20.400,LUNANEW_USDT,1h,STOCH_RSI,0.639600,0.759229,-1.323435,0.452247,1.143181,-2.632631,...,1.450960,-18.658140,5.037532,1.450960,-18.050832,5.037532,1.450960,-17.730483,5.037532,1.450960
1722,1970-01-20 15:47:38.400,ASTRA_USDT,1h,STOCH_RSI,0.026800,0.028579,-0.439303,0.726421,0.090803,-0.734322,...,2.814883,-6.094354,2.996488,2.814883,-6.440099,2.996488,2.814883,-6.754864,2.996488,2.814883
130,1970-01-20 15:47:45.600,DEGO_USDT,4h,Pattern_Trend,1.571000,1.493250,0.364203,0.178109,0.508884,0.543931,...,3.842073,1.071054,0.178109,3.842073,0.714761,0.178109,3.842073,0.378051,0.178109,3.842073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,2023-08-17 07:00:00.000,STMXUSDT,1h,STOCH_RSI,0.005638,0.005838,-0.222467,0.506183,0.204420,-0.546123,...,0.204420,0.000000,0.506183,0.204420,0.000000,0.506183,0.204420,0.000000,0.506183,0.204420
825,2023-08-17 08:00:00.000,PHAUSDT,1h,STOCH_RSI,0.088000,0.091254,-0.334433,0.288523,0.144262,0.000000,...,0.144262,0.000000,0.288523,0.144262,0.000000,0.288523,0.144262,0.000000,0.288523,0.144262
970,2023-08-17 08:00:00.000,AMPUSDT,1h,STOCH_RSI,0.002190,0.002251,-0.282140,0.163841,0.286721,0.000000,...,0.286721,0.000000,0.163841,0.286721,0.000000,0.163841,0.286721,0.000000,0.163841,0.286721
886,2023-08-17 08:00:00.000,DFUSDT,1h,STOCH_RSI,0.035400,0.038183,-0.493475,0.000000,0.288602,0.000000,...,0.288602,0.000000,0.000000,0.288602,0.000000,0.000000,0.288602,0.000000,0.000000,0.288602


In [6]:
import pandas as pd
from datetime import timedelta
from tqdm.auto import tqdm

def get_file(ticker):
    ''' Find files buy ticker names, file names can be in different formats '''
    try:
        tmp_df_1h = pd.read_pickle(f'../optimizer/ticker_dataframes/{ticker}_1h.pkl')
        tmp_df_4h = pd.read_pickle(f'../optimizer/ticker_dataframes/{ticker}_4h.pkl')
    except FileNotFoundError:
        pass
    else:
        return tmp_df_1h, tmp_df_4h
    
    try:
        tmp_df_1h = pd.read_pickle(f'../optimizer/ticker_dataframes/{ticker[:-4]}-{ticker[-4:]}_1h.pkl')
        tmp_df_4h = pd.read_pickle(f'../optimizer/ticker_dataframes/{ticker[:-4]}-{ticker[-4:]}_4h.pkl')
    except FileNotFoundError:
        pass
    else:
        return tmp_df_1h, tmp_df_4h
    
    try:
        tmp_df_1h = pd.read_pickle(f'../optimizer/ticker_dataframes/{ticker[:-4]}-{ticker[-4:]}-SWAP_1h.pkl')
        tmp_df_4h = pd.read_pickle(f'../optimizer/ticker_dataframes/{ticker[:-4]}-{ticker[-4:]}-SWAP_4h.pkl')
    except FileNotFoundError:
        pass
    else:
        return tmp_df_1h, tmp_df_4h
    
    return None, None


def create_train_df(df, type, target_offset, first, last, step):
    ''' Create train dataset from signal statistics and ticker candle data'''
    train_df = pd.DataFrame()
    tickers = df['ticker'].unique()
    
    for ticker in tqdm(tickers):
        signal_df = df[df['ticker'] == ticker]
        times = signal_df['time']
        
        tmp_df_1h, tmp_df_4h = get_file(ticker)

        for t in times.to_list():
            pass_cycle = False
            row = tmp_df_1h.loc[tmp_df_1h['time'] == t, :].reset_index(drop=True)
            
            for i in range(first, last + step, step):
                time_prev = t + timedelta(hours= -i)
                try:
                    row_tmp = tmp_df_1h.loc[tmp_df_1h['time'] == time_prev, :].reset_index(drop=True)
                    row_tmp.columns = [c + f'_prev_{i}' for c in row_tmp.columns]
                except IndexError:
                    pass_cycle = True
                    break
                row = pd.concat([row, row_tmp.iloc[:,1:]], axis=1)
                
            if pass_cycle:
                continue
            
            time_next = t + timedelta(hours=target_offset)
            if type == 'buy':
                target = tmp_df_1h.loc[tmp_df_1h['time'] == time_next, 'high'].reset_index(drop=True)
            else:
                target = tmp_df_1h.loc[tmp_df_1h['time'] == time_next, 'low'].reset_index(drop=True)

            target.name = 'target'
            rows = pd.concat([row, target], axis=1)
            
            if train_df.shape[0] == 0:
                train_df = rows
            else:
                train_df = pd.concat([train_df, rows])
    
    return train_df.reset_index(drop=True)

# for how long time (in hours) we want to predict
target_offset = 24
# first previous data point to collect for model training (value represents number of hours before signal point)
first = 1
# last previous data point to collect for model training (value represents number of hours before signal point)
last = 10
# step of previous data points collecting (total number of points to collect is (last - first + step) / step)
step = 1
# dataset with the signal statistics
df = pd.read_pickle('signal_stat/buy_stat_1h.pkl')
# dataset for model train
train_df = create_train_df(df, 'buy', target_offset, first, last, step)
train_df.head()


100%|██████████| 302/302 [00:16<00:00, 18.01it/s]


,time,open,high,low,close,volume,open_prev_1,high_prev_1,low_prev_1,close_prev_1,...,high_prev_8,low_prev_8,close_prev_8,volume_prev_8,open_prev_9,high_prev_9,low_prev_9,close_prev_9,volume_prev_9,target
0,2023-07-15 05:00:00,30336.45,30348.00,30288.70,30288.70,831.44144,30328.77,30390.90,30286.00,30336.45,...,30770.95,30050.00,30091.83,13453.45116,31213.99,31214.00,30648.00,30688.64,6225.48626,30335.00
1,2023-07-24 22:00:00,29124.25,29157.15,29102.00,29117.99,852.34591,29039.75,29137.65,29000.00,29124.26,...,29330.69,29222.34,29269.99,1486.61624,29178.00,29383.11,29116.16,29324.01,4638.01236,29277.95
2,2023-07-15 05:00:00,30336.45,30348.00,30288.70,30288.70,831.44144,30328.77,30390.90,30286.00,30336.45,...,30770.95,30050.00,30091.83,13453.45116,31213.99,31214.00,30648.00,30688.64,6225.48626,30335.00
3,2023-08-02 20:00:00,1829.11,1839.30,1825.65,1831.43,17174.02850,1842.42,1847.53,1821.21,1829.12,...,1860.00,1854.42,1854.71,7972.05580,1857.08,1858.23,1853.66,1857.38,12243.35140,1859.12
4,2023-08-07 21:00:00,1815.12,1831.33,1813.63,1821.90,16893.75390,1815.59,1818.83,1805.95,1815.12,...,1837.39,1833.50,1837.38,4040.73550,1833.32,1836.13,1832.55,1835.63,3994.09660,1871.43
